In [4]:
!pip install pyautogen langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.37
    Uninstalling langchain-core-0.3.37:
      Successfully uninstalled langchain-core-0.3.37


In [5]:
import autogen

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [6]:
# Configure the API settings (ensure you replace with your actual OpenAI API key)
config_list = [
    {
        "model": "gpt-4o",  # Use "gpt-3.5-turbo" if GPT-4 is unavailable
        "api_key": "*******************************************",  # Replace with your OpenAI API key
    }
]

In [13]:
from autogen import AssistantAgent, UserProxyAgent

In [11]:
# Define the Developer Agent that generates Python code
developer = AssistantAgent(
    name="Developer",
    llm_config={"config_list": config_list}
)

In [16]:
# Define the Reviewer Agent that reviews and refines the code provided by the Developer

reviewer = AssistantAgent(
    name="Reviewer",
    llm_config={"config_list": config_list}
)

In [17]:
# Define the Manager Agent that assigns the initial task
manager = UserProxyAgent(
    name="Manager",
    human_input_mode="NEVER",  # Operates autonomously
    max_consecutive_auto_reply=3,  # Limits consecutive auto replies
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    llm_config={"config_list": config_list}
)

In [36]:
# Global variable to store final code
final_code = {"content": ""}

In [37]:
# Define the conversation flow
def conversation_flow():
    global final_code

    # Step 1: Manager gives task to Developer
    manager.initiate_chat(
        developer,
        message="Write a Python function to check if a given number is prime."
    )

    # Step 2: Developer writes the code and sends it to Reviewer
    response = developer.initiate_chat(
        reviewer,
        message="Here is my function for checking if a number is prime. Can you review and suggest improvements?"
    )

    # Step 3: Reviewer evaluates the code and provides feedback
    review_response = reviewer.initiate_chat(
        developer,
        message="I will review your code and provide suggestions if necessary."
    )
    # Capture the final reviewed function
    final_code["content"] = review_response



In [38]:
# Start the conversation
conversation_flow()

Manager (to Developer):

Write a Python function to check if a given number is prime.

--------------------------------------------------------------------------------
Developer (to Manager):

To check if a given number is prime, we need to determine if it has any divisors other than 1 and itself. A number \( n \) is prime if:

1. It is greater than 1.
2. No number from 2 to \(\sqrt{n}\) (inclusive) divides \( n \) without leaving a remainder.

Here's a Python function that implements this logic:

```python
def is_prime(number):
    """Check if a number is prime."""
    if number <= 1:
        return False
    if number <= 3:
        return True
    if number % 2 == 0 or number % 3 == 0:
        return False
    i = 5
    while i * i <= number:
        if number % i == 0 or number % (i + 2) == 0:
            return False
        i += 6
    return True

# Test the function
number_to_test = 29
print(f"Is {number_to_test} a prime number? {is_prime(number_to_test)}")
```

This function beg

In [53]:
final_code["content"]

ChatResult(chat_id=None, chat_history=[{'content': 'I will review your code and provide suggestions if necessary.', 'role': 'assistant', 'name': 'Reviewer'}, {'content': "Thank you for being here to review the code. Please let me know which specific code or task you would like me to work on or review. I'll be happy to assist with that.", 'role': 'user', 'name': 'Developer'}, {'content': "Please let me know the specific code or task you need assistance with, and I'll be happy to help with the review or provide guidance on that.", 'role': 'assistant', 'name': 'Reviewer'}, {'content': "I don't have a specific code or task in progress at the moment. However, if you have a request for a code-related task or need assistance with a particular problem, feel free to let me know, and I'll be happy to help. Whether it's about solving a programming problem, writing a script, or reviewing some code, I'm here to assist you.", 'role': 'user', 'name': 'Developer'}, {'content': 'Great! Since there is n